# Python中函数是一等对象
## 一等对象定义
#### 1.在运行时创建
#### 2.能赋值给变量或数据结构中的元素
#### 3.能作为参数传递
#### 4.能作为运行结果返回


In [3]:
def factorial(n):
    '''return n!'''
    return 1 if n < 2 else n * factorial(n-1)
print(factorial(42))
print(factorial.__doc__)
print(type(factorial))

1405006117752879898543142606244511569936384000000000
return n!
<class 'function'>


In [4]:
# 函数可以赋值
fact = factorial
fact(5)

120

In [5]:
# list(Iterable)，map返回Iterable的生成器，map接受的是一个元素处理函数和Iterable对象。
list(map(fact, range(1,11)))

[1, 2, 6, 24, 120, 720, 5040, 40320, 362880, 3628800]

## 高阶函数
高阶函数就是可接受函数作为参数或作为结果返回的函数，因此map、filter、sorted、reduce等都是高阶函数。

#### Python3中map、filter、reduce都有相关的更好实现方式,map、filter返回迭代器。

reduce可使用多种内置函数替代，如需要sum归约则使用sum(Iterable)，类似的还有all(Iterable)、any(Iterable)等

In [6]:
# 仅奇数用于计算阶乘
print(list(map(factorial, filter(lambda x : x % 2,range(6)))))
# 可被推导替代
print([factorial(x) for x in range(6) if x % 2])

[1, 6, 120]
[1, 6, 120]


In [9]:
from functools import reduce
from operator import add
# sum归约
print(reduce(add, range(100)))
# sum归约使用内置函数
print(sum(range(100)))

4950
4950


##  匿名函数
python的句法使得lambda函数定义体只能使用春表达式，因此不能出现赋值和while、try等结构，因此匿名函数除了用于作为参数传递给高阶函数外很少使用。
#### lambda句法只是语法糖，与def一样，lambda表达式也会创建函数对象。
## 可调用函数（callable）
()其实是调用运算符，可以应用到可调用对象上，判断对象是否callable可以使用内置的callable()函数确认。python中可调用对象通常有7种：  
1.__用户自定义函数__：def或lambda创建。  
2.__内置函数__：使用C实现的函数，如len、time.strftime  
3.__内置方法__：C实现的方法如：dict.get，有点类似Java静态函数  
4.__类定义体中定义的函数__  
5.__类__：调用类时运行\__new__创建实例，然后运行\__init__初始化实例返回。  
6.__类的实例__：若类定义了\__call__方法，则其实例可以调用，如function的实例。  
7.__生成器函数__：使用yield的函数或方法，调用时返回生成器对象。
## 自定义可调用对象
Python中任何对象都变得可调用，只需要在类实中实现定义的\__call__方法。往往可以利用\__call__机制返回可调用对象实现需要内部维护状态的函数（如缓存）。
## 函数内省
#### 内省是指计算机程序在运行时（Run time）检查对象（Object）类型的一种能力，通常也可以称作运行时类型检查。
除了\__doc__，函数还有很多属性，如：  
1.\__annotations__（dict）用于存储函数参数和返回值的注解。  
2.\__call__（method-wrapper）即可调用运算符()的实现,即可调用协议的具体实现。  
3.\__code__：（code）编译为字节码的函数元数据和函数定义体。  
4.\__defaults__:（tuple）形参默认值。  
5.\__get__(method-wrapper)只读描述符协议的实现。
6.\__globals__(dict)函数所在模块的全局变量。  
7.\__kwdefaults__(dict)仅限关键字形参默认值。
## 函数参数
python的参数处理机制极为灵活，python3进一步提供了仅限关键字参数

In [1]:
# name作为位置或关键参数必须指定。另外，无默认值定位参数必须在前。
# content作为位置参数捕获除name外的所有位置参数（首先被打包为元组）。
# cls作为仅限关键字参数，必须指定，但这里已有默认值可调用时可不指定。
# attrs是仅限关键自参数，捕获除cls外所有关键字参数（首先被打包为dict）。
# 由于cls和attrs出现在*之后，因此表示为仅限关键字参数，如果不想有*content用于支持不定数量的位置参数可以仅放置*。
def tag(name='a', *content, cls=None, **attrs):
    '''生成一个或多个html标签'''
    if cls is not None:
        attrs['class'] = cls
    if attrs:
        # 快速由Iterable(生成器表达式)生成str。
        attr_str = ''.join(' %s="%s"' % (attr, value) for attr, value in sorted(attrs.items()))
    else:
        attr_str = ''
    if content:
        return '\n'.join('<%s%s>%s</%s>' % (name, attr_str, c, name) for c in content)
    else:
        return '<%s%s />' % (name, attr_str)

In [12]:
# 仅name赋值
tag('br')

'<br />'

In [13]:
# name被赋值，content捕获位置参数'hello'
tag('br', 'hello')

'<br>hello</br>'

In [14]:
# name被赋值，content捕获参数'hello'，仅限关键字参数attrs捕获关键字参数id
tag('p', 'hello', id=33)

'<p id="33">hello</p>'

In [15]:
# name被赋值，content捕获参数'hello'，仅限关键字参数cls被赋值
tag('p', 'hello', 'world', cls='sidebar')

'<p class="sidebar">hello</p>\n<p class="sidebar">world</p>'

In [16]:
# 位置或关键字参数name被赋值，仅限关键字参数attrs捕获关键字参数content（注意不是给形参content赋值，形参content代表捕获除name外其余全部位置参数）
tag(content='testing', name='img')

'<img content="testing" />'

In [18]:
my_tag = {'name': 'img', 'title': 'Sunset Boulevard', 'src': 'sunset.jpg', 'cls': 'framed'}
# 位置或关键字参数name被赋值,仅限关键字参数cls被赋值，仅限关键字参数attrs捕获my_tag中其余全部关键字参数。
tag(**my_tag)

'<img class="framed" src="sunset.jpg" title="Sunset Boulevard" />'

## 获取函数参数信息
像IDE或是框架有时需要函数内省以得到函数对象的相关信息，从而完成诸如参数检查、网络请求参数自动映射等功能。例如HTTP微框架Bobo，可以通过对函数参数的内省自动从请求中获取参数并传递。

In [ ]:
import bobo
# bobo通过函数对象的__defaults__（元组）属性得知函数的定位参数和关键字参数的默认值，仅限关键字参数默认值在__kwdefaults__中。
# 函数参数名称保存在__code__（code）属性中，它是一个引用。
@bobo.query('/')
def hello('/'):
    return 'Hello %s' % person

In [5]:
# 位置或关键字参数默认值 
print(tag.__defaults__)
# 仅限关键字参数默认值 
print(tag.__kwdefaults__)
print(tag.__code__)
# 函数的参数名称+函数内局部变量名
print(tag.__code__.co_varnames)
# 函数的位置或关键字参数个数（不含*的变长参数）
print(tag.__code__.co_argcount)
# 函数的仅限关键字参数个数（不含**的变长参数）
print(tag.__code__.co_kwonlyargcount)

('a',)
{'cls': None}
<code object tag at 0x000001DC8F42CF60, file "<ipython-input-1-9c394d44f298>", line 6>
('name', 'cls', 'content', 'attrs')
1
1


# 获取函数签名
如果想通过函数内省去得到参数名称及相应默认值的对应关系，需要利用co_varnames、co_argcount、co_kwonlyargcount，因为co_varnames存在局部变量名。同时需要\__defaults__、\__kwdefaults__得到相关默认值信息，且需对于位置或关键字参数，需要从后向前扫描才能把参数和默认值对应起来。
## 函数签名
利用内置的签名模块可以方便获取函数的参数信息。  
利用内置的签名模块可以对函数的形参进行重新绑定。  
框架或IDE可利用这些工具获取函数信息以验证代码。

In [11]:
from inspect import signature
# 得到inspect.Signature对象
sig = signature(tag)
print(repr(sig))
# inspect.Signature.parameters是OrderedDict，存储参数名和对应属性包装对象inspect.Parameter(param)
for name, param in sig.parameters.items():
    print(param.kind, ":", name, '=', param.default)

<Signature (name='a', *content, cls=None, **attrs)>
POSITIONAL_OR_KEYWORD : name = a
VAR_POSITIONAL : content = <class 'inspect._empty'>
KEYWORD_ONLY : cls = None
VAR_KEYWORD : attrs = <class 'inspect._empty'>


In [13]:
my_tag = {'name': 'img', 'title': 'Sunset Boulevard', 'src': 'sunset.jpg', 'cls': 'framed'}
# 重新使用my_tag函数签名进行形参绑定，my_tag中name被位置或关键字参数name捕获，cls被仅限关键字捕获，其余关键字参数被attrs关键字参数字典捕获。
bounds_args = sig.bind(**my_tag)
print(repr(bounds_args))
# 展示绑定的参数情况BoundArguments.arguments是一个OrderedDict
for name, value in bounds_args.arguments.items():
    print(name, "=", value)

<BoundArguments (name='img', cls='framed', attrs={'title': 'Sunset Boulevard', 'src': 'sunset.jpg'})>
name = img
cls = framed
attrs = {'title': 'Sunset Boulevard', 'src': 'sunset.jpg'}


# 函数注解
函数声明中各个参数可以在':'后增加注解表达式，可以是类和字符串，如果参数有默认值则注解放在参数名和'='之间  
获取参数注解可以通过属性包装对象inspect.Parameter(param),即上述param.annotation。  
函数注解对python解释器没有任何意义，仅提供函数相关信息，因此其作用主要为IDE的静态类型检查和类似于bobo的web框架用于自动类型转化。
# 支持函数式编程的包
python可以轻易支持函数式编程风格，在函数式编程中，常常需要将算术运算符当做函数使用。python的operator模块为多个运算符提供了对应的函数，从而避免编写lambda a,b : a*b这样的匿名函数。

In [15]:
from functools import reduce
from operator import mul
# 不用递归实现连乘
def fact(n):
    return reduce(mul, range(1, n+1))

#### operator的itemgetter能替代从序列中取出特定元素，利用attrgetter能得到对象的属性。上述两个方法可自行构建函数并返回可调用对象，能避免定义匿名函数从而编写出函数式风格的程序。  
itemgetter使用[]运算符，因此支持任何实现\__getitem__的类，如序列、映射等。  
attrgetter可创建用于提取对象属性的函数，传入的参数名如果含有'.'还可获取嵌套的属性。

In [16]:
# 使用itemgetter读取序列元素
meta_data = [
    ('Tokyo', 'JP', 36.933, (35.689722, 139.691667)),
    ('Delhi NCR', 'IN', 21.935, (28.613889, 77.208889)),
    ('Mexico City', 'MX', 20.142, (19.433333, -99.133333)),
    ('New York-Newark', 'US', 20.104, (40.808611, -74.020386)),
    ('Sao Paulo', 'BR', 19.649, (-23.547778, -46.635833))
]
from operator import itemgetter
# 按照国家排序
for city in sorted(meta_data, key=itemgetter(1)):
    print(city)

('Sao Paulo', 'BR', 19.649, (-23.547778, -46.635833))
('Delhi NCR', 'IN', 21.935, (28.613889, 77.208889))
('Tokyo', 'JP', 36.933, (35.689722, 139.691667))
('Mexico City', 'MX', 20.142, (19.433333, -99.133333))
('New York-Newark', 'US', 20.104, (40.808611, -74.020386))


In [19]:
# 创建从序列中获取元素方式函数，得到国家、城市
cc_name = itemgetter(1, 0) # 等价于 lambda x : x[1], x[0]
for city in meta_data:
    print(cc_name(city))

('JP', 'Tokyo')
('IN', 'Delhi NCR')
('MX', 'Mexico City')
('US', 'New York-Newark')
('BR', 'Sao Paulo')


In [26]:
from collections import namedtuple
from operator import attrgetter
Car = namedtuple('Car', 'name engine tire')
Engine = namedtuple('Engine', 'piston, power')
super_car = Car('RR', Engine(piston=8, power=589), tire='Michelin')

power = attrgetter('name', 'engine.power')
print(power(super_car))

('RR', 589)


# 冻结参数
和attrgetter方法相对应的还有methodecaller，会创建函数调用方法，调用传入对象的相应函数,同时可冻结参数值。但methodecaller只能用于指定类的函数并对该类对象产生作用。
#### functools.partial可对任意函数进行参数冻结。

In [30]:
from operator import methodcaller
s = 'the time has come'
replace = methodcaller('replace',' ','-')
replace(s)

'the-time-has-come'

In [33]:
from functools import partial
triple = partial(mul, 3)
print(triple)
list(map(triple, range(1, 10))) # map中的函数实参只能接收一个参数，可使用参数冻结方式。

functools.partial(<built-in function mul>, 3)


[3, 6, 9, 12, 15, 18, 21, 24, 27]